In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import pickle
import cv2

In [2]:
# load model
model_pca = pickle.load(open('./model/pca_50.pickle','rb'))
model_svm = pickle.load(open('./model_svm.pickle','rb'))
model_mean = pickle.load(open('./model/mean_preprocess.pickle','rb'))
haar = cv2.CascadeClassifier('./model/haarcascade_frontalface_default.xml')

In [3]:
label = ['Male','Female']
font = cv2.FONT_HERSHEY_SIMPLEX

In [4]:
def pipeline(img):

    #img = cv2.imread('./data/male_000281.jpg')
    # step-1 : convert into gray scale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # step-2: apply haar and extract face
    faces = haar.detectMultiScale(gray,1.3,5)
    # crop and draw rectangle
    for x,y,w,h in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),1)
        roi = gray[y:y+h,x:x+h]
        if roi.shape[0] < 100:
            roi_re = cv2.resize(roi,(100,100),cv2.INTER_CUBIC)
        else:
            roi_re = cv2.resize(roi,(100,100),cv2.INTER_AREA)
        # flatten image
        roi_flat = roi_re.reshape(1,-1)
        # normalize 
        roi_norm = roi_flat / 255.0
        # subract with mean
        roi_mean = roi_norm - model_mean
        # transform pca
        x_pca_50 = model_pca.transform(roi_mean)
        # apply to the model 
        res = model_svm.predict_proba(x_pca_50)
        # get gender 
        gender = res.argmax()
        score = res[0][gender]
        lab = label[gender]
        text = "%s : %0.2f"%(lab,score)
        # put in image
        cv2.putText(img,text,(x,y),font,1,(0,255,0),3)
    
    return img

In [6]:
img = pipeline('./data/male_000281.jpg')

TypeError: src is not a numpy array, neither a scalar

In [ ]:
plt.imshow(img)

In [16]:
# video
cap = cv2.VideoCapture('./data/_Thor_ Ragnarok_ Official Trailer.mp4')

while True:
    ret, frame = cap.read()
    #frame = cv2.resize(frame,())
    
    if ret == False:
        break
        
    frame = pipeline(frame)
    
    cv2.imshow('Gender Detect',frame)
    if cv2.waitKey(20) == 27:
        break
        
cv2.destroyAllWindows()
cap.release()